In [1]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

import os
import glob
import requests
import pandas as pd
from PIL import Image
from array import array
from tqdm.auto import tqdm

from data.api_keys import *

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
computervision_client


In [2]:
encompass_images = glob.glob("./data/encompass/*/*.jpg")
camp_images = glob.glob("./data/camp/*/*.jpg")
print(f"encompass: {len(encompass_images)}\tcamp: {len(camp_images)}")

encompass: 21588	camp: 10273


In [3]:
# test_image = encompass_images[0]
# test_image

In [4]:
# image_data = open(test_image, "rb").read()
# analyze_url = endpoint + "vision/v3.2/analyze?visualFeatures=Categories,Objects,Tags&model-version=latest"
# headers = {'Ocp-Apim-Subscription-Key': subscription_key,
#            'Content-Type': 'application/octet-stream'}
# response = requests.post(
#     analyze_url, headers=headers, data=image_data)
# response.raise_for_status()

# analysis = response.json()
# print(analysis)

In [11]:
class Analyzer:
    def __init__(self, endpoint, subscription_key):
        self.analyze_url = endpoint + "vision/v3.2/analyze?visualFeatures=Categories,Objects,Tags&model-version=latest"
        self.headers = {'Ocp-Apim-Subscription-Key': subscription_key,
                       'Content-Type': 'application/octet-stream'}
        
        self.col_names = ["dataset", "ID", "number", "fname", "categories", "tags", "objects"]
        self.df = pd.DataFrame([], columns=self.col_names)
        
    def run(self, image_path, index):
        meta_ = image_path.split("/")
        meta = [meta_[2], meta_[3], int(index), meta_[-1]]
        
        image = open(image_path, "rb").read()
        try:
            response = requests.post(
                self.analyze_url, headers=self.headers, data=image)
            response.raise_for_status()

            results = response.json()
            self.append_results(meta, results)
        except:
            print(f"unexpected error: {image_path}")
    
    
    def append_results(self, meta, results):
        dataset, ID, number, fname = meta
        categories = results['categories']
        tags = results['tags']
        objects = results['objects']
        
        self.df = self.df.append({self.col_names[0]: dataset, 
                                  self.col_names[1]: ID, 
                                  self.col_names[2]: number, 
                                  self.col_names[3]: fname, 
                                  self.col_names[4]: categories, 
                                  self.col_names[5]: tags, 
                                  self.col_names[6]: objects}, ignore_index=True)
        

In [12]:
analyzer = Analyzer(endpoint, subscription_key)

for index, image in enumerate(tqdm(encompass_images)):
    analyzer.run(image, index)


  0%|          | 0/21588 [00:00<?, ?it/s]

unexpected error: ./data/encompass/sns002/sns002_0019.jpg
unexpected error: ./data/encompass/sns003/sns003_0108.jpg
unexpected error: ./data/encompass/sns003/sns003_0346.jpg
unexpected error: ./data/encompass/sns003/sns003_0765.jpg
unexpected error: ./data/encompass/sns003/sns003_0890.jpg
unexpected error: ./data/encompass/sns004/sns004_0028.jpg
unexpected error: ./data/encompass/sns018/sns018_0043.jpg
unexpected error: ./data/encompass/sns021/sns021_0052.jpg


In [13]:
for index, image in enumerate(tqdm(camp_images)):
    analyzer.run(image, index)


  0%|          | 0/10273 [00:00<?, ?it/s]

unexpected error: ./data/camp/esther_chaenny/2019-01-15 18.45.32 1957276243801672398_2996699855.jpg
unexpected error: ./data/camp/jjeongstagram/2017-05-06 12.16.10 1508442350228961105_477117679.jpg


In [14]:
analyzer.df.to_csv("results/image_information_azure.csv", index=False)

In [15]:
analyzer.df

,dataset,ID,number,fname,categories,tags,objects
0,encompass,sns002,0,sns002_0001.jpg,"[{'name': 'abstract_nonphoto', 'score': 0.5}, ...","[{'name': 'text', 'confidence': 0.999978721141...","[{'rectangle': {'x': 196, 'y': 537, 'w': 412, ..."
1,encompass,sns002,1,sns002_0002.jpg,"[{'name': 'others_', 'score': 0.09765625}, {'n...","[{'name': 'text', 'confidence': 0.999947249889...","[{'rectangle': {'x': 124, 'y': 529, 'w': 400, ..."
2,encompass,sns002,2,sns002_0003.jpg,"[{'name': 'people_group', 'score': 0.41796875}...","[{'name': 'human face', 'confidence': 0.998585...","[{'rectangle': {'x': 330, 'y': 637, 'w': 106, ..."
3,encompass,sns002,3,sns002_0004.jpg,"[{'name': 'drink_', 'score': 0.99609375}]","[{'name': 'text', 'confidence': 0.993455529212...",[]
4,encompass,sns002,4,sns002_0005.jpg,"[{'name': 'others_', 'score': 0.015625}, {'nam...","[{'name': 'text', 'confidence': 0.999690890312...","[{'rectangle': {'x': 612, 'y': 424, 'w': 266, ..."
...,...,...,...,...,...,...,...
31846,camp,yooseoni_i,10268,2019-08-09 23.19.32 2106717950559097719_402754...,"[{'name': 'people_portrait', 'score': 0.839843...","[{'name': 'person', 'confidence': 0.9958564639...","[{'rectangle': {'x': 98, 'y': 0, 'w': 647, 'h'..."
31847,camp,yooseoni_i,10269,2019-07-09 23.05.17 2084242731647054588_402754...,"[{'name': 'people_', 'score': 0.85546875}]","[{'name': 'person', 'confidence': 0.9965835809...","[{'rectangle': {'x': 171, 'y': 7, 'w': 520, 'h..."
31848,camp,yooseoni_i,10270,2019-08-01 23.11.34 2100915741011333070_402754...,"[{'name': 'people_young', 'score': 0.46875}]","[{'name': 'person', 'confidence': 0.9971476793...","[{'rectangle': {'x': 93, 'y': 73, 'w': 576, 'h..."
31849,camp,yooseoni_i,10271,2019-09-12 23.33.51 2131367530286139018_402754...,"[{'name': 'people_young', 'score': 0.44921875}]","[{'name': 'person', 'confidence': 0.9986907243...","[{'rectangle': {'x': 0, 'y': 4, 'w': 283, 'h':..."
